<a href="https://www.kaggle.com/code/hatemamine/scotuslexglue-preprocesslongtextwithkeybert?scriptVersionId=114577228" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install keybert --quiet
from keybert import KeyBERT
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas 
import numpy as np 
import re
import matplotlib
import seaborn as sb 
import time

In [ ]:
#get the number of words 
def getNumberOfWords(df):
    maxl=[]
    for i in range(len(df)):
        x= df["ATexte"][i].split()
        if len(x)< 3: print(i)
        maxl.append(len(x))
    print("Max value element : ", max(maxl))
    print("Min value element : ", min(maxl))
    return maxl
    

In [ ]:
def plot_NumberOfWord(maxl, dataset_name):
    fig = plt.figure()
    ax1 = plt.axes()
    ax1.set_ylabel('number of words')
    ax1.set_title('number of words for each instance '+dataset_name)
    ind = np.arange(len(maxl))
    line, =ax1.plot(ind,maxl)
    plt.axhline(y=512, color="red", linestyle="--")
    fig.savefig(dataset_name+".png")
    plt.show()

In [ ]:
def plotNumberSampleforeachCLASS(df, dataset_name):
    sb.countplot(df.MotsCles)
    matplotlib.pyplot.xlabel("Number of sample for each class Distribution "+dataset_name)
    matplotlib.pyplot.savefig(dataset_name+'distribution.png')
    plt.show()

In [ ]:
def PreprocessLongTextwithKeyBERT(df, n_gram):
    kw_model = KeyBERT()
    start_time = time.time()
    for i in range(len(df)):
        tmpnewtext=""
        x= df["ATexte"][i].split()
        #if len(x) < 513 : continue 
        textgen=""
        for j in range(len(x)):
            textgen =textgen +" "+x[j]
            if j ==0:continue # for not process the first empty string
            if (j % 512 ==0) or (len(x)-1==j): 
                keygen = kw_model.extract_keywords(textgen, keyphrase_ngram_range=(n_gram, n_gram), stop_words='english', use_mmr=True, diversity=0.9, top_n=int(max(1, round((512*512)/(len(x)*1), 0))))
                textgen=""
                for y in range(len(keygen)):
                    tmpnewtext=tmpnewtext +" "+keygen[y][0]
                #print(keygen[y][0])
        df["ATexte"][i]=tmpnewtext
        #break
    print("--- %s seconds ---" % (time.time() - start_time))
    return df

In [ ]:
dataset = load_dataset("lex_glue", "scotus")
DATASET_NAME='SCOTUS'
for x in dataset.keys():
    #get the train dataset
    dict = {'ATexte': dataset[x]["text"], 'MotsCles': dataset[x]["label"]}
    df = pandas.DataFrame(dict)
    df["ATexte"]= df["ATexte"].str.lower()
    for i in range(len(df)):
        df["ATexte"][i] = re.sub(r'[^a-z ]', '', df["ATexte"][i])
    print(x,len(df),"sample")
    plotNumberSampleforeachCLASS(df, DATASET_NAME+x)
    plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x)
    df.to_csv("/kaggle/working/"+DATASET_NAME+x+".csv")
    df = PreprocessLongTextwithKeyBERT(df, 3)
    df = PreprocessLongTextwithKeyBERT(df, 1)
    plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x+"KeyBERT")
    df.to_csv("/kaggle/working/"+DATASET_NAME+x+"KeyBERT.csv")